## API

좌표계 변환 사이트 : https://epsg.io/transform#s_srs=3857&t_srs=4326&x=14817776.5552511&y=4688953.0631259

In [ ]:
import requests as rq
import urllib.parse as parse
import urllib.request as request  
import json
import pandas as pd
from pandas.io.json import json_normalize
    
url = "http://api.vworld.kr/req/data"   
service_key = "677BCD51-6065-3FBD-80C2-2E178E20DCDD"
request_urls = []
# version : 요청 서비스 버전
# request : 요청 서비스 오퍼레이션
# crs : 응답좌표계
queryParams = '?' + parse.urlencode({ parse.quote_plus('service') : 'data'  
                               , parse.quote_plus('request') : 'GetFeature'
                               , parse.quote_plus('version') : '2.0'
                               , parse.quote_plus('key') : service_key
                               , parse.quote_plus('format') : 'json'
                               , parse.quote_plus('errorFormat') : 'json'
                               , parse.quote_plus('size') : '10'
                               , parse.quote_plus('page') : '1'
                               , parse.quote_plus('data') : 'LT_L_TRKROAD' # LT_C_ADSIDO_INFO
                               , parse.quote_plus('geomfilter') : 'BOX(122.7392578, 32.9902355, 133.1103517, 38.7712163)' # 한반도 전체지역(좌표계:EPSG:3857): 13663271.680031825,3894007.9689600193,14817776.555251127,4688953.0631258525 을 EPSG:4326으로 변환
                               , parse.quote_plus('geometry') : 'true'
                               , parse.quote_plus('attribute') : 'true'
                            #    , parse.quote_plus('columns') : 'cos_nam'
                               , parse.quote_plus('crs') : 'EPSG:4326'
                                })   

res = rq.get(url, queryParams)

# test용 url
# url = "https://api.vworld.kr/req/data?key={}&request=GetFeature&service=data&version=2.0&format=json&size=1000&page=1&geometry=false&attribute=true&crs=EPSG:3857&geomfilter=BOX(13663271.680031825,3894007.9689600193,14817776.555251127,4688953.0631258525)&data=LT_C_ADSIDO_INFO".format(service_key)
# res = rq.get(url)


# print(res.url)
# print(res.status_code)
# print(res.text)

json_data = json.loads(res.text)
# print(json.dumps(json.loads(res.text), sort_keys=True, indent=4, ensure_ascii = False))

df = pd.DataFrame(json_data)
df = pd.json_normalize(json_data['response']['result']['featureCollection']['features'])
df = df.iloc[:, 3:]
df

,geometry.coordinates,properties.cat_nam,properties.lnk_nam,properties.cos_num,properties.cos_nam,properties.cos_lvl,properties.len_tim,properties.leng_lnk,properties.comment
0,"[[[127.7846719856947, 35.16237732803098], [127...",둘레길링크,하동호-삼화실,11코스,지리산둘레길,하,←4시간 9.3Km→,9342.42811403,
1,"[[[127.65931897718815, 35.18070376552526], [12...",둘레길링크,원부춘-가탄,15코스,지리산둘레길,상,←7시간30분 12.6Km→,13006.1001180,
2,"[[[127.75974281501351, 35.10290440337887], [12...",둘레길링크,하동읍-서당,13코스,지리산둘레길,하,←2시간30분 7.1Km→,7015.43251329,
3,"[[[127.58708082167367, 35.21776381053337], [12...",둘레길링크,목아재-당재,17코스,지리산둘레길,중,←3시간 7.8Km→,7909.00839247,
4,"[[[127.60337954723201, 35.42996623453473], [12...",둘레길링크,인월-금계,3코스,지리산둘레길,중,←8시간 19.3Km→,20457.8570286,
5,"[[[127.53082624461805, 35.440650005656984], [1...",둘레길링크,운봉-인월,2코스,지리산둘레길,중,←4시간 9.4Km→,9372.14217937,
6,"[[[127.4454921074571, 35.39082602777359], [127...",둘레길링크,주천-운봉,1코스,지리산둘레길,중,←6시간 14.3Km→,14564.6135652,
7,"[[[127.4635552642758, 35.26121956128309], [127...",둘레길링크,방광-산동,21코스,지리산둘레길,상,←6시간 13Km→,12770.7762270,
8,"[[[127.51500422178758, 35.205644472010604], [1...",둘레길링크,송정-오미,18코스,지리산둘레길,중,←5시간30분 9.2Km→,9875.17087729,
9,"[[[127.5728033822975, 35.20605879726299], [127...",둘레길링크,가탄-송정,16코스,지리산둘레길,상,←6시간30분 11.3Km→,9964.21062247,


In [ ]:
df['properties.comment']

0    
1    
2    
3    
4    
5    
6    
7    
8    
9    
Name: properties.comment, dtype: object

In [ ]:
df.drop('properties.comment', axis=1, inplace=True)

In [ ]:
df.columns = ['좌표', '산책로구분', '명칭', '코스번호', '코스명', '코스난이도', '소요시간', '코스길이(km)']

In [ ]:
df

,좌표,산책로구분,명칭,코스번호,코스명,코스난이도,소요시간,코스길이(km)
0,"[[[127.7846719856947, 35.16237732803098], [127...",둘레길링크,하동호-삼화실,11코스,지리산둘레길,하,←4시간 9.3Km→,9342.42811403
1,"[[[127.65931897718815, 35.18070376552526], [12...",둘레길링크,원부춘-가탄,15코스,지리산둘레길,상,←7시간30분 12.6Km→,13006.1001180
2,"[[[127.75974281501351, 35.10290440337887], [12...",둘레길링크,하동읍-서당,13코스,지리산둘레길,하,←2시간30분 7.1Km→,7015.43251329
3,"[[[127.58708082167367, 35.21776381053337], [12...",둘레길링크,목아재-당재,17코스,지리산둘레길,중,←3시간 7.8Km→,7909.00839247
4,"[[[127.60337954723201, 35.42996623453473], [12...",둘레길링크,인월-금계,3코스,지리산둘레길,중,←8시간 19.3Km→,20457.8570286
5,"[[[127.53082624461805, 35.440650005656984], [1...",둘레길링크,운봉-인월,2코스,지리산둘레길,중,←4시간 9.4Km→,9372.14217937
6,"[[[127.4454921074571, 35.39082602777359], [127...",둘레길링크,주천-운봉,1코스,지리산둘레길,중,←6시간 14.3Km→,14564.6135652
7,"[[[127.4635552642758, 35.26121956128309], [127...",둘레길링크,방광-산동,21코스,지리산둘레길,상,←6시간 13Km→,12770.7762270
8,"[[[127.51500422178758, 35.205644472010604], [1...",둘레길링크,송정-오미,18코스,지리산둘레길,중,←5시간30분 9.2Km→,9875.17087729
9,"[[[127.5728033822975, 35.20605879726299], [127...",둘레길링크,가탄-송정,16코스,지리산둘레길,상,←6시간30분 11.3Km→,9964.21062247


In [ ]:
df.shape

(10, 8)

In [ ]:
df.iloc[0, 0][0]

[[127.7846719856947, 35.16237732803098],
 [127.78495444044457, 35.16231452313443],
 [127.785226073734, 35.16222942648314],
 [127.78538962202042, 35.162141721836036],
 [127.78551831762823, 35.16205696115819],
 [127.78559264970795, 35.16212812162641],
 [127.78578367854303, 35.16233881341112],
 [127.78581007213899, 35.16240913279958],
 [127.78585878826327, 35.16263208981091],
 [127.7858786266333, 35.16269705583014],
 [127.78590061948077, 35.16275545587463],
 [127.78595318381578, 35.16283227344787],
 [127.78605299277768, 35.162919119251285],
 [127.78615781484436, 35.16298567443719],
 [127.78642404695333, 35.16310015926874],
 [127.78655205321003, 35.16314995861261],
 [127.78684230054142, 35.16327750838683],
 [127.78690342417822, 35.163309572043374],
 [127.78700299377216, 35.16332782588867],
 [127.78712316668377, 35.16330842115153],
 [127.78726282591268, 35.16327412604295],
 [127.78737455001516, 35.163237477737106],
 [127.78769760857732, 35.16298799156121],
 [127.78790544745429, 35.162813931

In [ ]:
len(df.iloc[0, 0][0])

672

In [ ]:
df.iloc[0, 0][0][::10]

[[127.7846719856947, 35.16237732803098],
 [127.78590061948077, 35.16275545587463],
 [127.78726282591268, 35.16327412604295],
 [127.78847348137538, 35.161801599121866],
 [127.78828677262, 35.16096433112535],
 [127.78838187446928, 35.16023203824643],
 [127.79022354623255, 35.15884979249095],
 [127.79215543189027, 35.1583079877457],
 [127.79318011894179, 35.157844847111456],
 [127.79428611449701, 35.15745089464824],
 [127.79492119844562, 35.15681747241501],
 [127.7956022366583, 35.15668375901921],
 [127.79621990196652, 35.15685939541948],
 [127.7973050768587, 35.15689896570628],
 [127.79765961559778, 35.15637453233562],
 [127.7983211820424, 35.15626035022554],
 [127.80160549147591, 35.15471011652035],
 [127.8010409413816, 35.15354909914739],
 [127.80226780113767, 35.15047998149733],
 [127.80343449915587, 35.14952549282749],
 [127.8028612621999, 35.1467847643476],
 [127.80188612960838, 35.14510868177155],
 [127.80057444634214, 35.14450897873229],
 [127.80008792111013, 35.14394177911066],
 

In [ ]:
len(df.iloc[0, 0][0][::10])

68

In [ ]:
for i in range(len(df['좌표'])):
    df.iloc[i, 0] = df.iloc[i, 0][0][::10]

In [ ]:
df

,좌표,산책로구분,명칭,코스번호,코스명,코스난이도,소요시간,코스길이(km)
0,"[[127.7846719856947, 35.16237732803098], [127....",둘레길링크,하동호-삼화실,11코스,지리산둘레길,하,←4시간 9.3Km→,9342.42811403
1,"[[127.65931897718815, 35.18070376552526], [127...",둘레길링크,원부춘-가탄,15코스,지리산둘레길,상,←7시간30분 12.6Km→,13006.1001180
2,"[[127.75974281501351, 35.10290440337887], [127...",둘레길링크,하동읍-서당,13코스,지리산둘레길,하,←2시간30분 7.1Km→,7015.43251329
3,"[[127.58708082167367, 35.21776381053337], [127...",둘레길링크,목아재-당재,17코스,지리산둘레길,중,←3시간 7.8Km→,7909.00839247
4,"[[127.60337954723201, 35.42996623453473], [127...",둘레길링크,인월-금계,3코스,지리산둘레길,중,←8시간 19.3Km→,20457.8570286
5,"[[127.53082624461805, 35.440650005656984], [12...",둘레길링크,운봉-인월,2코스,지리산둘레길,중,←4시간 9.4Km→,9372.14217937
6,"[[127.4454921074571, 35.39082602777359], [127....",둘레길링크,주천-운봉,1코스,지리산둘레길,중,←6시간 14.3Km→,14564.6135652
7,"[[127.4635552642758, 35.26121956128309], [127....",둘레길링크,방광-산동,21코스,지리산둘레길,상,←6시간 13Km→,12770.7762270
8,"[[127.51500422178758, 35.205644472010604], [12...",둘레길링크,송정-오미,18코스,지리산둘레길,중,←5시간30분 9.2Km→,9875.17087729
9,"[[127.5728033822975, 35.20605879726299], [127....",둘레길링크,가탄-송정,16코스,지리산둘레길,상,←6시간30분 11.3Km→,9964.21062247


In [ ]:
df = df.astype('string')

In [ ]:
df.dtypes

좌표          string
산책로구분       string
명칭          string
코스번호        string
코스명         string
코스난이도       string
소요시간        string
코스길이(km)    string
dtype: object

## Save

In [ ]:
df.to_csv('../data/산책로_2013.csv', index=False, encoding='cp949')